In [4]:
import pandas as pd
import glob

In [18]:
# 设置文件夹路径
folder_path = "./data/train_sets"

# 使用glob获取所有csv文件路径
csv_files = glob.glob(folder_path + "/*.csv")

In [26]:
dataframe = pd.read_csv(csv_files[6])
dataframe.head(5)

,1727426633.8038833,"[-0.35215026514299286, 1.5, 0, 0, 0.0, -3.8159508734208876, 0]",1032,776,1033,820
0,1.727427e+09,"[-0.35173052295873375, 1.5, 0, 0, 0.0, -3.8167...",1032,776,1033,820
1,1.727427e+09,"[-0.35173052295873375, 1.5, 0, 0, 0.0, -3.8167...",1033,776,1033,820
2,1.727427e+09,"[-0.38205049227460397, 1.5, 0, 0, 0.0, -3.9380...",1033,776,1033,821
3,1.727427e+09,"[-0.38205049227460397, 1.5, 0, 0, 0.0, -3.9380...",1033,777,1033,821
4,1.727427e+09,"[-0.3792429712530818, 1.5, 0, 0, 0.0, -3.93184...",1033,776,1032,820


        1727423078.0681956  \
0             1.727423e+09   
1             1.727423e+09   
2             1.727423e+09   
3             1.727423e+09   
4             1.727423e+09   
...                    ...   
186024                 NaN   
186025                 NaN   
186026                 NaN   
186027                 NaN   
186028                 NaN   

       [1.5, -0.7102062845817496, 0, 0, 0.0, 4.494500607817749, 0]    362  \
0       [1.5, -0.7460282147996353, 0, 0, 0.0, 4.633800...           362.0   
1       [1.5, -0.7460282147996353, 0, 0, 0.0, 4.633800...           362.0   
2       [1.5, -0.6951662102800806, 0, 0, 0.0, 4.439460...           362.0   
3       [1.5, -0.6951662102800806, 0, 0, 0.0, 4.439460...           363.0   
4       [1.5, -0.6301619610176807, 0, 0, 0.0, 4.191203...           363.0   
...                                                   ...             ...   
186024                                                NaN             NaN   
186025                 